In [1]:
import pandas as pd
import sqlite3

In [2]:
def get_most_common_pit_stop_lap(conn: str) -> pd.DataFrame:
    query = '''
    WITH MostCommonPitStopLap AS (
        SELECT
            races.year,
            drivers.driverId,
            pit_stops.lap AS most_common_pitstop_lap,
            ROW_NUMBER() OVER (PARTITION BY drivers.driverId, races.year ORDER BY COUNT(pit_stops.lap) DESC) AS lap_rank
        FROM
            drivers
        JOIN
            pit_stops ON drivers.driverId = pit_stops.driverId
        JOIN
            races ON pit_stops.raceId = races.raceId
        WHERE
            races.year != '2023'
        GROUP BY
            drivers.driverId, races.year, pit_stops.lap
    )
    SELECT
        driverId,
        year,
        most_common_pitstop_lap
    FROM
        MostCommonPitStopLap
    WHERE
        lap_rank = 1;
    '''
    
    pit_stop_lap = pd.read_sql_query(query, conn)

    return pit_stop_lap


In [3]:
def get_pit_stop_stats_in_miliseconds(conn: str) -> pd.DataFrame:
    """
    Retrieve pit stop statistics (average, minimum, and maximum pit stop times) for each driver and year.
    
    :param conn: The database connection.
    :return: DataFrame containing driverId, year, avg_pit_time, min_pit_stop_time, and max_pit_stop_time.
    """
    query = '''
    SELECT
        drivers.driverId,
        races.year,
        MIN(pit_stops.milliseconds) AS min_pit_stop_time,
         AVG(pit_stops.milliseconds) AS avg_pit_time,
        MAX(pit_stops.milliseconds) AS max_pit_stop_time
    FROM
        drivers 
    JOIN
        pit_stops ON drivers.driverId = pit_stops.driverId
    JOIN
        races ON pit_stops.raceId = races.raceId
    WHERE
        races.year != '2023'
    GROUP BY
        drivers.driverId, races.year;
    '''

    df = pd.read_sql_query(query, conn)

    return df
